In [43]:
import pandas as pd

ci_d2_df = pd.read_csv("../data/e_and_c_benchmark/counting_impact_pop_do2.csv")
ci_d2_df = ci_d2_df[ci_d2_df.generation == 100]

ci_d3_df = pd.read_csv("../data/e_and_c_benchmark/counting_impact_pop_do3.csv")
ci_d3_df = ci_d3_df[ci_d3_df.generation == 100]

In [44]:
# build the accuumulared fata

import pandas as pd

# =============================================================================
# d2
# =============================================================================
df_d2_data = {
    #"ramdpom_population": [],
    "seed": [],
    "seed_type": [],
    "algorithm_name": [],
    "problem_name": [],
}

for index, row in ci_d2_df.iterrows():
    seed_type = row.seed_type
    seed_list = []
    if "+" in seed_type and not "c" in seed_type: #edges
        seed_list = ["traceID_1", "traceID_2"]
    elif not "+" in seed_type: #single points (edges or center)
        seed_list = ["traceID_1"]
    elif "+" in seed_type and "c" in seed_type: #edges + center
        seed_list = ["traceID_1", "traceID_2", "traceID_3"]
    #df_d2_data["ramdpom_population"].append( row[ ["traceID_"+str((i+1)) for i in range(len(seed_list), 91)] ].sum() ) #popsize is always 91!
    df_d2_data["seed"].append( row[ seed_list ].sum() )
    df_d2_data["seed_type"].append( seed_type )
    df_d2_data["algorithm_name"].append( row.algorithm_name )
    df_d2_data["problem_name"].append( row.problem_name )

df_2d = pd.DataFrame(data=df_d2_data)
print(df_2d.head())

# =============================================================================
# d3
# =============================================================================
df_d3_data = {
    #"ramdpom_population": [],
    "seed": [],
    "seed_type": [],
    "algorithm_name": [],
    "problem_name": [],
}

for index, row in ci_d3_df.iterrows():
    seed_type = row.seed_type
    seed_list = []
    if "+" in seed_type and not "c" in seed_type: #edges
        seed_list = ["traceID_1", "traceID_2", "traceID_3"]
    elif not "+" in seed_type: #single points (edges or center)
        seed_list = ["traceID_1"]
    elif "+" in seed_type and "c" in seed_type: #edges + center
        seed_list = ["traceID_1", "traceID_2", "traceID_3", "traceID_4"]
    #df_d3_data["ramdpom_population"].append( row[ ["traceID_"+str((i+1)) for i in range(len(seed_list), 91)] ].sum() ) #popsize is always 91!
    df_d3_data["seed"].append( row[ seed_list ].sum() )
    df_d3_data["seed_type"].append( seed_type )
    df_d3_data["algorithm_name"].append( row.algorithm_name )
    df_d3_data["problem_name"].append( row.problem_name )

df_3d = pd.DataFrame(data=df_d3_data)
print(df_3d.head())


        

       seed seed_type algorithm_name problem_name
0  0.095334         r          NSGA2       MACO_b
1  0.132082         r          NSGA2       MACO_b
2  0.159971         r          NSGA2       MACO_b
3  0.190238         r          NSGA2       MACO_b
4  0.000000         r          NSGA2       MACO_b
   seed seed_type algorithm_name problem_name
0   0.0         r          NSGA2          UF8
1   0.0         r          NSGA2          UF8
2   0.0         r          NSGA2          UF8
3   0.0         r          NSGA2          UF8
4   0.0         r          NSGA2          UF8


In [65]:
import plotly.express as px

def plot_CI_data(filtered_df, problem_name, n_obj):
    fig = px.box(
        filtered_df.loc[(filtered_df.problem_name.str.contains(problem_name))],
        color="seed_type",
        facet_col="problem_name",
        facet_row="algorithm_name",
        labels={
                "value": "counting impact",
                "variable": "",
                "percentage_of_optimal_genes": "% optimal genes",
                "seed" : "seed ind"
            },
        #title=problem_name+" d_o="+str(n_obj),
        facet_col_spacing=0.0,
        facet_row_spacing=0.0,
        #set font size to 12:
        
        )
    fig.update_layout(
        height=300,
        width=180*len(filtered_df.problem_name.unique()),
        margin=dict(l=0, r=0, t=0, b=0),
        font=dict(size=13)
    )
    fig.for_each_annotation(lambda a: a.update( text=a.text.removeprefix("problem_name=") ))#remove the prefix
    fig.for_each_annotation(lambda a: a.update( text=a.text.removeprefix("algorithm_name=") ))#remove the prefix
    fig.write_image("../figures/paper/pop_impact/impact_plot_"+problem_name+"d_o="+str(n_obj)+".pdf")
    fig.show()


# plot_CI_data( df_2d, "ZDT", 2 )
# plot_CI_data( df_2d, "MACO", 2 )
# plot_CI_data( df_2d, "UF", 2 )

# plot_CI_data( df_3d, "UF", 3 )
# plot_CI_data( df_3d, "DTLZ", 3 )

d2_data = df_2d[ (df_2d.problem_name != "ZDT4") & (df_2d.problem_name != "ZDT6") & (df_2d.problem_name != "MACO_w=shallow") ]
plot_CI_data( d2_data, "", 2 )
plot_CI_data( df_3d, "", 3 )

In [9]:
import pandas as pd
import plotly.express as px

def plot_CI_data(filtered_df):
    fig = px.box(
        filtered_df,
        color="seed_type",
        facet_col="problem_name",
        facet_row="algorithm_name",
        labels={
                "value": "counting impact",
                "variable": "",
                "percentage_of_optimal_genes": "% optimal genes",
                "traceID_1" : "seed ind"
            },
        title="",
        height=700, width=2100
        )
    #fig.write_image("../figures/rosenbrock_genome_vs_fitness_impact.pdf")
    fig.show()

gen = 100

ci_df = ci_d2_df#pd.concat([ci_d2_df, ci_d3_df])
ci_df = ci_df[ci_df.generation == gen]
ci_df['random_population'] = ci_df[["traceID_"+str(i) for i in range(2, 92)]].sum(axis=1) #91 = pop_size

#current_df = ci_df[['traceID_1', 'random_population', 'traceID_m', 'algorithm_name', 'problem_name', "seed_type"]]
current_df = ci_df[['traceID_1', 'algorithm_name', 'problem_name', "seed_type"]] #first seed only
plot_CI_data(current_df)


ci_df = ci_d3_df#pd.concat([ci_d2_df, ci_d3_df])
ci_df = ci_df[ci_df.generation == gen]
ci_df['random_population'] = ci_df[["traceID_"+str(i) for i in range(2, 92)]].sum(axis=1) #91 = pop_size

#current_df = ci_df[['traceID_1', 'random_population', 'traceID_m', 'algorithm_name', 'problem_name', "seed_type"]]
current_df = ci_df[['traceID_1', 'algorithm_name', 'problem_name', "seed_type"]] #first seed only
plot_CI_data(current_df)

/var/folders/wp/_16y9p6573n_95q3dky8ktjm0000gp/T/ipykernel_72194/1961882137.py:26: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



/var/folders/wp/_16y9p6573n_95q3dky8ktjm0000gp/T/ipykernel_72194/1961882137.py:35: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



# Seed Heritage of PF

## 2D Data

In [1]:
import pandas as pd

trace_data_d2 = pd.read_csv("../data/e_and_c_benchmark/counting_impact_inds_do2_gen100.csv")
fitness_data_d2 = pd.read_csv("../data/e_and_c_benchmark/fitness_and_ranks_do2.csv")
fitness_data_d2 = fitness_data_d2[(fitness_data_d2.generation == 100) & (fitness_data_d2.goldberg_rank == 0)]
data_d2 = pd.merge(left=trace_data_d2, right=fitness_data_d2, how="inner")
del trace_data_d2
del fitness_data_d2

# print(data_d2)

In [2]:
# some helper methods

import plotly.graph_objects as go

def get_2d_scatterplot(data, seedID) -> go.scatter:
    return go.Scatter(
            x=data["f_1"],
            y=data["f_2"],
            mode="markers",
            marker=dict(
                color=data[seedID],
                colorscale=px.colors.sequential.algae,
                colorbar=dict(thickness=20, bordercolor='white', title="Impact"),
                cmin=0,
                cmax=1,
                opacity=0.5
            ),
            showlegend=False,
        )

def get_2d_pf_fig(problem) -> go.scatter:
    pf = problem._calc_pareto_front()
    return go.Scatter(
                x=pf[:, 0],
                y=pf[:, 1],
                mode="markers",
                marker=dict(size=3, color="grey"),
                showlegend=False
            )

In [29]:
# Plot all of the 2d one seed data
import plotly.express as px
import plotly.graph_objects as go

from test_setup import problems

def plot_2d_data(data, problem_name):
    data = data[data.problem_name == problem_name]
    data = data.drop(columns=["problem_name"])
    data = data[data.seed_type != "r"]

    fig = px.scatter(
        data,
        x="f_1",
        y="f_2",
        facet_col="seed_type",
        facet_row="algorithm_name",
        color="traceID_1",
        color_continuous_scale=px.colors.sequential.algae,
        #color_continuous_midpoint=0.0,
        range_color=[0, 1],
        title=problem_name,
        labels={
            "traceID_1": "Impact"
        },
        facet_col_spacing=0.0,
        facet_row_spacing=0.0
    )


    #add PF plot to each sub plot
    for j in range(1, len(data['seed_type'].unique())+1):#rows
        for i in range(1, len(data['algorithm_name'].unique())+1):#cols
            fig.add_trace(get_2d_pf_fig(problems[problem_name]), row=i, col=j)
    fig.update_layout(title=problem_name, height=400, width=800,
        margin=dict(l=0, r=0, t=25, b=0),
    )
    fig.for_each_annotation(lambda a: a.update( text=a.text.removeprefix("seed_type=") ))#remove the prefix
    fig.for_each_annotation(lambda a: a.update( text=a.text.removeprefix("algorithm_name=") ))#remove the prefix
    fig.write_image("../figures/paper/impact_pf/single_seed/single_seed_impact_pf_"+problem_name+".pdf")

data = data_d2[["generation", "traceID_1", "f_1", "f_2", "problem_name", "algorithm_name", "seed_type"]]
data = data[(data.seed_type == "e1") | (data.seed_type == "e2") | (data.seed_type == "c")]
problem_names = data.problem_name.unique()
for problem_name in problem_names:
    plot_2d_data(data, problem_name)


In [36]:
# Plot all of the 2d comboned seed data
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


def plot_2d_data(data, problem_name, algorithm_name):
    data = data[data.algorithm_name == algorithm_name]
    data = data[data.problem_name == problem_name]
    data = data.drop(columns=["problem_name"])
    data = data[data.seed_type != "r"]

    fig = make_subplots(rows=1, cols=3,
                        subplot_titles=("e1", "e2", "c"),
                        horizontal_spacing = 0.06,
                        #vertical_spacing=0.06
        )
    for i, seedID in enumerate(["traceID_1", "traceID_2", "traceID_3"]):
        fig.add_trace(get_2d_pf_fig(problems[problem_name]), row=1, col=i+1) #add the true PF
        fig.add_trace(get_2d_scatterplot(data, seedID), row=1, col=i+1)
    fig.update_layout(title=problem_name+" "+algorithm_name, height=200, width=800,
                      margin=dict(l=0, r=0, t=40, b=0),
    )
    #fig.show()
    fig.write_image("../figures/paper/impact_pf/combined_seeds/combined_seed_impact_"+problem_name+"_"+algorithm_name+".pdf")


data = data_d2[["generation", "traceID_1", "traceID_2", "traceID_3", "f_1", "f_2", "problem_name", "algorithm_name", "seed_type"]]
data = data[data.seed_type == "e1+e2+c"]
problem_names = data.problem_name.unique()
for problem_name in problem_names:
    plot_2d_data(data, problem_name, "NSGA2")
    plot_2d_data(data, problem_name, "MOEAD")

## 3D Data

In [17]:
import pandas as pd

trace_data_d3 = pd.read_csv("../data/e_and_c_benchmark/counting_impact_inds_do3_gen100.csv")
fitness_data_d3 = pd.read_csv("../data/e_and_c_benchmark/fitness_and_ranks_do3.csv")
fitness_data_d3 = fitness_data_d3[(fitness_data_d3.generation == 100) & (fitness_data_d3.goldberg_rank == 0)]
data_d3 = pd.merge(left=trace_data_d3, right=fitness_data_d3, how="inner")
del trace_data_d3
del fitness_data_d3

# print(data_d3)

In [21]:
# some helper methods

import plotly.graph_objects as go

def get_3d_scatterplot(data, seedID) -> go.scatter3d:
    return go.Scatter3d(
            x=data["f_1"],
            y=data["f_2"],
            z=data["f_3"],
            mode="markers",
            marker=dict(
                color=data[seedID],
                colorscale=px.colors.sequential.algae,
                colorbar=dict(thickness=20, bordercolor='white', title="Impact"),
                cmin=0,
                cmax=1,
                opacity=0.5
            ),
            showlegend=False,
        )

def get_3d_pf_fig(problem) -> go.scatter3d:
    pf = problem._calc_pareto_front()
    return go.Scatter3d(
                x=pf[:, 0],
                y=pf[:, 1],
                z=pf[:, 2],
                mode="markers",
                marker=dict(size=3, color="grey"),
                showlegend=False
            )


#The scene camera:
camera = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=1.5, y=1.5, z=1.5)
)

In [39]:
# Plot all of the 3d one seed data
from test_setup import problems

def plot_3d_data(data, problem_name, algorithm_name):
    data = data[data.algorithm_name == algorithm_name]
    data = data[data.problem_name == problem_name]
    data = data.drop(columns=["problem_name"])
    data = data[data.seed_type != "r"]

    fig = make_subplots(
        rows=1,
        cols=4,
        subplot_titles=("e1", "e2", "e3", "c"),
        start_cell="top-left",
        specs=[
            [{"type": "scatter3d"}, {"type": "scatter3d"}, {"type": "scatter3d"}, {"type": "scatter3d"}],
        ],
        horizontal_spacing = 0.0
    )

    for i, seed_type in enumerate(data.seed_type.unique()): #columns
        fig.update_scenes(
            camera=camera,
            row=1, col=i+1
        )
        fig.add_trace(get_3d_scatterplot(data[data.seed_type == seed_type], "traceID_1"), row=1, col=i+1)
        fig.add_trace(get_3d_pf_fig(problems[problem_name]), row=1, col=i+1) #add the true PF
        
    fig.update_layout(title=problem_name+" "+algorithm_name, height=400, width=1600,
        margin=dict(l=0, r=0, t=35, b=0),
    )

    fig.write_image("../figures/paper/impact_pf/single_seed/single_seed_impact_p_"+problem_name+"_"+algorithm_name+".pdf")

data = data_d3[["generation", "traceID_1", "f_1", "f_2", "f_3", "problem_name", "algorithm_name", "seed_type"]]
data = data[(data.seed_type == "e1") | (data.seed_type == "e2") | (data.seed_type == "e3") |(data.seed_type == "c")]
problem_names = data.problem_name.unique()
for problem_name in problem_names:
    plot_3d_data(data, problem_name, "NSGA2")
    plot_3d_data(data, problem_name, "MOEAD")


In [40]:
# Plot all of the 3d combined seed data
from plotly.subplots import make_subplots

from test_setup import problems


def plot_3d_data(data, problem_name, algorithm_name):
    data = data[data.algorithm_name == algorithm_name]
    data = data[data.problem_name == problem_name]
    data = data.drop(columns=["problem_name"])

    fig = make_subplots(
        rows=1,
        cols=4,
        subplot_titles=("e1", "e2", "e3", "c"),
        start_cell="top-left",
        specs=[
            [{"type": "scatter3d"}, {"type": "scatter3d"}, {"type": "scatter3d"}, {"type": "scatter3d"}], 
        ],
        horizontal_spacing = 0.0
    )

    for i, seedID in enumerate(["traceID_1", "traceID_2", "traceID_3", "traceID_4"]):
        fig.add_trace(get_3d_pf_fig(problems[problem_name]), row=1, col=i+1) #add the true PF
        fig.add_trace(get_3d_scatterplot(data, seedID), row=1, col=i+1)
        fig.update_scenes(camera=camera, row=1, col=i+1)
    fig.update_layout(title=problem_name+" "+algorithm_name, height=400, width=1600,
        margin=dict(l=0, r=0, t=35, b=0),
    )
    #fig.show()
    fig.write_image("../figures/paper/impact_pf/combined_seeds/combined_seed_impact_"+problem_name+"_"+algorithm_name+".pdf")


data = data_d3[["generation", "traceID_1", "traceID_2", "traceID_3", "traceID_4", "f_1", "f_2", "f_3", "problem_name", "algorithm_name", "seed_type"]]
data = data[data.seed_type == "e1+e2+e3+c"]

problem_names = data.problem_name.unique()
for problem_name in problem_names:
    plot_3d_data(data, problem_name, "NSGA2")
    plot_3d_data(data, problem_name, "MOEAD")